<a href="https://colab.research.google.com/github/shawrkz/Final/blob/main/Plotly_Dash_Air_Quality_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import dash
from dash import dcc
from dash import html
import plotly.graph_objects as go
import pandas as pd
import requests
import datetime
from dash.dependencies import Input, Output
from dash.exceptions import PreventUpdate

# --- 1. Fetch Data from OpenAQ API ---
def fetch_air_quality_data(city, parameter='PM2.5', date_from=None, date_to=None):
    """
    Fetches air quality data for a specific city and parameter from the OpenAQ API.

    Args:
        city (str): The name of the city (case-insensitive).
        parameter (str, optional): The air quality parameter to fetch (e.g., 'PM2.5', 'O3'). Defaults to 'PM2.5'.
        date_from (str, optional):  The start date for the data in 'YYYY-MM-DD' format. Defaults to None.
        date_to (str, optional): The end date for the data in 'YYYY-MM-DD' format. Defaults to None.

    Returns:
        pandas.DataFrame: A DataFrame containing the air quality data, or None if an error occurs.
    """
    url = f'https://api.openaq.org/v2/measurements?city={city}&parameter={parameter}'
    if date_from:
        url += f'&date_from={date_from}'
    if date_to:
        url += f'&date_to={date_to}'
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()
        if not data['results']:
            print(f"No data found for {city}, parameter: {parameter}")
            return pd.DataFrame()
        df = pd.DataFrame(data['results'])
        df['date_utc'] = pd.to_datetime(df['date']['utc'])
        df = df[['date_utc', 'value', 'unit', 'location']]  # Select relevant columns
        return df
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from OpenAQ: {e}")
        return None
    except KeyError:
        print(f"Error: Unexpected data format from OpenAQ API for {city}, parameter: {parameter}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

def get_top_locations(df, n=5):
    """
    Gets the top n locations from the dataframe.

    Args:
        df (pd.DataFrame): The input dataframe.
        n (int): The number of top locations to return.

    Returns:
        pd.DataFrame: A DataFrame containing the top n locations.
    """
    if df is None or df.empty:
        return pd.DataFrame()
    location_counts = df['location'].value_counts()
    top_locations = location_counts.head(n).index.tolist()
    return df[df['location'].isin(top_locations)]

# --- 2. Create the Dash App ---
app = dash.Dash(__name__)

# --- 3. Define the Layout ---
app.layout = html.Div(children=[
    html.H1(children='Air Quality Dashboard', style={'textAlign': 'center'}),

    html.Div(children='Select a city and date range to visualize air quality data.', style={'textAlign': 'center'}),

    # City and Parameter selection dropdowns
    html.Div([
        html.Div([
            html.Label('Select City:', style={'display': 'block', 'margin-bottom': '5px'}),
            dcc.Dropdown(
                id='city-dropdown',
                options=[
                    {'label': 'Los Angeles', 'value': 'Los Angeles'},
                    {'label': 'New York', 'value': 'New York'},
                    {'label': 'London', 'value': 'London'},
                    {'label': 'Paris', 'value': 'Paris'},
                    {'label': 'Tokyo', 'value': 'Tokyo'},
                    {'label': 'Delhi', 'value': 'Delhi'},
                    {'label': 'Chicago', 'value': 'Chicago'},
                    {'label': 'Houston', 'value': 'Houston'},
                    {'label': 'Phoenix', 'value': 'Phoenix'},
                    {'label': 'Philadelphia', 'value': 'Philadelphia'}
                ],
                value='Los Angeles',  # Default city
                style={'width': '200px'}
            ],
            style={'display': 'inline-block', 'margin-right': '20px', 'margin-left': '20px'}
        ]),
        html.Div([
            html.Label('Select Parameter:', style={'display': 'block', 'margin-bottom': '5px'}),
            dcc.Dropdown(
                id='parameter-dropdown',
                options=[
                    {'label': 'PM2.5', 'value': 'PM2.5'},
                    {'label': 'PM10', 'value': 'PM10'},
                    {'label': 'O3', 'value': 'O3'},
                    {'label': 'NO2', 'value': 'NO2'},
                    {'label': 'SO2', 'value': 'SO2'},
                    {'label': 'CO', 'value': 'CO'}
                ],
                value='PM2.5',  # Default parameter
                style={'width': '200px'}
            ],
            style={'display': 'inline-block', 'margin-right': '20px'}
        ]),
        html.Div([
            html.Label('Select Start Date:', style={'display': 'block', 'margin-bottom': '5px'}),
            dcc.DatePickerSingle(
                id='start-date-picker',
                display_format='YYYY-MM-DD',
                style={'width': '200px'},
                date=datetime.datetime(2023, 1, 1)
            ),
        ], style={'display': 'inline-block', 'margin-right': '20px'}),
        html.Div([
            html.Label('Select End Date:', style={'display': 'block', 'margin-bottom': '5px'}),
            dcc.DatePickerSingle(
                id='end-date-picker',
                display_format='YYYY-MM-DD',
                style={'width': '200px'},
                date=datetime.datetime.now()
            ),
        ], style={'display': 'inline-block'})
    ]),

    # Graph to display air quality data
    dcc.Graph(id='air-quality-graph', style={'width': '80%', 'margin': 'auto'}),

    # Pie charts
    html.Div([
        html.Div([
            dcc.Graph(id='location-pie-chart-1'),
        ], style={'display': 'inline-block', 'width': '30%'}),
        html.Div([
            dcc.Graph(id='location-pie-chart-2'),
        ], style={'display': 'inline-block', 'width': '30%'}),
        html.Div([
            dcc.Graph(id='location-pie-chart-3'),
        ], style={'display': 'inline-block', 'width': '30%'}),
    ], style={'width': '80%', 'margin': 'auto', 'display': 'flex', 'justifyContent': 'space-around'}),

    html.Div(id='data-source', style={'textAlign': 'center', 'margin-top': '10px', 'fontStyle': 'italic', 'fontSize': '12px'})
])

# --- 4. Define the Callbacks ---
@app.callback(
    Output('air-quality-graph', 'figure'),
    Output('location-pie-chart-1', 'figure'),
    Output('location-pie-chart-2', 'figure'),
    Output('location-pie-chart-3', 'figure'),
    Output('data-source', 'children'),
    [
        Input('city-dropdown', 'value'),
        Input('parameter-dropdown', 'value'),
        Input('start-date-picker', 'date'),
        Input('end-date-picker', 'date')
    ]
)
def update_graph(selected_city, selected_parameter, start_date, end_date):
    """
    Callback function to update the air quality graph and pie charts based on user selections.

    Args:
        selected_city (str): The city selected by the user.
        selected_parameter (str): The air quality parameter selected by the user.
        start_date (str): The start date selected by the user.
        end_date (str): The end date selected by the user.

    Returns:
        plotly.graph_objects.Figure: The updated Plotly figure.
        str: The data source attribution text.
    """
    # Format dates for the API if they are not None
    date_from_str = start_date if start_date else None
    date_to_str = end_date if end_date else None

    df = fetch_air_quality_data(selected_city, selected_parameter, date_from_str, date_to_str)

    if df is None or df.empty:
        # Return a blank plot with a message
        empty_fig = go.Figure(data=[go.Scatter()], layout=go.Layout(title=f"No data found for {selected_city} for {selected_parameter}"))
        return empty_fig, empty_fig, empty_fig, empty_fig, "Data provided by OpenAQ (no data available for selected parameters)"

    # Create the line chart
    line_chart = go.Figure(data=[
        go.Scatter(
            x=df['date_utc'],
            y=df['value'],
            mode='lines',
            marker=dict(color='blue'),
            name=selected_parameter
        )
    ])
    line_chart.update_layout(
        title=f"{selected_city} Air Quality - {selected_parameter}",
        xaxis_title="Date",
        yaxis_title=f"Value ({df['unit'].iloc[0]})",
        template='plotly_white'
    )

    # Create the pie charts
    top_locations_df = get_top_locations(df, n=5) # Get a df with only the top 5 locations.

    if top_locations_df is None or top_locations_df.empty:
        empty_pie = go.Figure(data=[go.Pie()], layout=go.Layout(title="No data"))
        return line_chart, empty_pie, empty_pie, empty_pie, "Data provided by OpenAQ"

    # Create three pie charts, each with a maximum of 5 locations
    pie_chart_1 = create_location_pie_chart(top_locations_df, title=f"{selected_city} - Top 5 Locations - 1")
    pie_chart_2 = create_location_pie_chart(top_locations_df, offset=5, title=f"{selected_city} - Top 5 Locations - 2")
    pie_chart_3 = create_location_pie_chart(top_locations_df, offset=10, title=f"{selected_city} - Top 5 Locations - 3")

    return line_chart, pie_chart_1, pie_chart_2, pie_chart_3, "Data provided by OpenAQ"

def create_location_pie_chart(df, offset=0, title=""):
    """
    Creates a pie chart of the top 5 locations.

    Args:
        df (pd.DataFrame): The input dataframe.
        offset (int): The offset for the locations.
        title (str): The title of the pie chart.

    Returns:
        plotly.graph_objects.Figure: The pie chart.
    """
    location_counts = df['location'].value_counts()
    # Get the top 5 locations
    top_locations = location_counts.head(5).index.tolist()
    # Filter the dataframe to only include the top 5 locations
    df_filtered = df[df['location'].isin(top_locations)]

    if df_filtered.empty:
        return go.Figure(data=[go.Pie()], layout=go.Layout(title=title))

    labels = df_filtered['location'].unique().tolist()
    values = [df_filtered[df_filtered['location'] == loc]['value'].mean() for loc in labels] # Use the filtered df

    pie_chart = go.Figure(data=[go.Pie(labels=labels, values=values, name="Location")],
                           layout=go.Layout(title=title))
    return pie_chart

# --- 5. Run the App ---
if __name__ == '__main__':
    app.run_server(debug=True)